# Validation

We primarily validate tax model input datasets by comparing against the IRS Survey of Incomes data releases. For each data year, we have access to between 1,000 and 2,000 individual statistical targets (e.g. "total taxable Social Security for filers with AGI between 30k and 40k"). For each dataset produced by this repo, we can attempt to reproduce these statistics.

There are valid reasons why datasets might not be able to reproduce SOI statistics:

1. The PUF sample size is too low (e.g. "estate losses" are in the tens of thousands for some AGI breakdowns).
2. The PUF is missing granular tax form information (e.g. some capital gains tax details that form the totals are not in the PUF data).
3. Datasets that are for a tax model produce different projected tax output variables than the PUF (e.g. Tax-Calculator or PolicyEngine might have a different EITC value than is reported in the PUF data).

Regardless, it's still useful to know because we should be able to reproduce most of the SOI statistics.

## Measuring quality

One way to measure the quality of fit against SOI targets might be to just take the mean relative deviation. However, the SOI statistics that fall under category (1) (and to some extent the others) tend to blow up this mean. Those SOI statistics might not be particularly important, and we want the quality of fit indicator to have some useful informational quality, so this is not ideal.

Instead, this validation exercise marks an SOI statistic as "OK" if the relative deviation is less than 5 percent, or if the absolute deviation is less than 1 million for filer count statistics and 1 billion for aggregate statistics.

With that definition, we can report to the percentage of SOI statistics (in the relevant year) that are "OK" for each dataset below.

In [1]:
from tax_microdata_benchmarking.utils.soi_replication import *
from tax_microdata_benchmarking.storage import STORAGE_FOLDER
from tax_microdata_benchmarking.datasets import *
import pandas as pd

INPUTS = STORAGE_FOLDER / "input"
OUTPUTS = STORAGE_FOLDER / "output"

puf_2015 = pd.read_csv(INPUTS / "puf_2015.csv")
tc_puf_2015 = pd.read_csv(OUTPUTS / "tc_puf_2015.csv")

soi_from_puf_2015 = compare_soi_replication_to_soi(
    puf_to_soi(puf_2015, 2015), 2015
)
soi_from_pe_puf_2015 = compare_soi_replication_to_soi(
    pe_to_soi(PUF_2015, 2015), 2015
)
soi_from_tc_puf_2015 = compare_soi_replication_to_soi(
    tc_to_soi(tc_puf_2015, 2015), 2015
)


def soi_statistic_passes_quality_test(df):
    # Relative error lower than this => OK
    RELATIVE_ERROR_THRESHOLD = 0.05

    # Absolute error lower than this for filer counts => OK
    COUNT_ABSOLUTE_ERROR_THRESHOLD = 1e6

    # Absolute error lower than this for aggregates => OK
    AGGREGATE_ABSOLUTE_ERROR_THRESHOLD = 1e9

    relative_error_ok = (
        df["Absolute relative error"] < RELATIVE_ERROR_THRESHOLD
    )
    absolute_error_threshold = np.where(
        df.Count,
        COUNT_ABSOLUTE_ERROR_THRESHOLD,
        AGGREGATE_ABSOLUTE_ERROR_THRESHOLD,
    )
    absolute_error_ok = df["Absolute error"] < absolute_error_threshold

    return relative_error_ok | absolute_error_ok


# 2021 datasets

puf_2021 = pd.read_csv(OUTPUTS / "puf_2021.csv")
tc_puf_2021 = pd.read_csv(OUTPUTS / "tc_puf_2021.csv")
tmd_2021 = pd.read_csv(OUTPUTS / "tmd_2021.csv")

soi_from_puf_2021 = compare_soi_replication_to_soi(
    puf_to_soi(puf_2021, 2021), 2021
)
soi_from_pe_puf_2021 = compare_soi_replication_to_soi(
    pe_to_soi(PUF_2021, 2021), 2021
)
soi_from_tc_puf_2021 = compare_soi_replication_to_soi(
    tc_to_soi(tc_puf_2021, 2021), 2021
)
soi_from_tmd_2021 = compare_soi_replication_to_soi(
    tc_to_soi(tmd_2021, 2021), 2021
)

dataset_soi_comparisons = [
    soi_from_puf_2015,
    soi_from_pe_puf_2015,
    soi_from_tc_puf_2015,
    soi_from_puf_2021,
    soi_from_pe_puf_2021,
    soi_from_tc_puf_2021,
    soi_from_tmd_2021,
]

for dataset in dataset_soi_comparisons:
    dataset["OK"] = soi_statistic_passes_quality_test(dataset)

dataset_names = [
    "PUF (2015)",
    "PE PUF (2015)",
    "TC PUF (2015)",
    "PUF (2021)",
    "PE PUF (2021)",
    "TC PUF (2021)",
    "TMD (2021)",
]

comparison_df = pd.DataFrame(
    {
        "Dataset": dataset_names,
        "SOI match score": [
            (df["OK"].mean() * 100).round(1) for df in dataset_soi_comparisons
        ],
    }
)

comparison_df

,Dataset,SOI match score
0,PUF (2015),95.2
1,PE PUF (2015),80.5
2,TC PUF (2015),89.4
3,PUF (2021),63.4
4,PE PUF (2021),70.7
5,TC PUF (2021),68.6
6,TMD (2021),71.2


Note that the pure PUF-derived datasets have lower scores than the PUF with reported tax output values. This is because the tax models (both of them) produced different estimates for tax variables, including the adjusted gross income which is used to bracket out some of the SOI statistics. This can mean that even if, for example, tax-exempt pension income is simply copied directly into the dataset, if the tax model produces different adjusted gross incomes for records then the tax-exempt pension income SOI statistics by AGI might be different than in the SOI releases.

## SOI score by variable

We can break out this score further, by variable.

In [2]:
soi_from_tmd_2021.to_csv(OUTPUTS / "soi_from_puf_tmd_2021.csv", index=False)

In [3]:
score_by_dataset = pd.DataFrame(
    {
        dataset_name: (dataset.groupby("Variable").OK.mean() * 100).round(1)
        for dataset_name, dataset in zip(
            dataset_names, dataset_soi_comparisons
        )
    }
).fillna(
    100
)  # Fillna because some variables aren't in the 2021 SOI releases.
score_by_dataset.sort_values("TMD (2021)")

,PUF (2015),PE PUF (2015),TC PUF (2015),PUF (2021),PE PUF (2021),TC PUF (2021),TMD (2021)
Variable,,,,,,,
adjusted_gross_income,88.1,60.3,84.1,19.9,38.2,28.5,41.4
income_tax_after_credits,100.0,40.0,72.5,42.7,31.7,35.4,42.7
employment_income,95.2,78.6,92.9,37.5,44.4,45.8,47.2
business_net_profits,100.0,95.2,97.6,55.6,52.8,55.6,52.8
rent_and_royalty_net_income,88.1,92.9,47.6,65.3,80.6,54.2,54.2
unemployment_compensation,100.0,100.0,100.0,65.3,58.3,56.9,56.9
capital_gains_gross,97.6,66.7,66.7,63.9,59.7,56.9,56.9
state_and_local_tax_deductions,54.2,52.1,56.2,41.7,52.1,60.4,62.5
taxable_income,100.0,45.0,90.0,45.1,67.1,52.4,64.6


In [10]:
soi_from_puf_2015[soi_from_puf_2015.Variable == "state_and_local_tax_deductions"]

,Variable,Filing status,AGI lower bound,AGI upper bound,Count,Taxable only,Full population,Value,SOI Value,Error,Absolute error,Relative error,Absolute relative error,OK
1211,state_and_local_tax_deductions,All,-inf,inf,False,False,True,5.416627e+11,352701327000,1.889614e+11,1.889614e+11,0.535755,0.535755,False
1212,state_and_local_tax_deductions,All,-inf,inf,False,True,False,5.177131e+11,344469879000,1.732432e+11,1.732432e+11,0.502927,0.502927,False
1213,state_and_local_tax_deductions,All,-inf,inf,True,False,True,4.416052e+07,42690831,1.469688e+06,1.469688e+06,0.034426,0.034426,True
1214,state_and_local_tax_deductions,All,-inf,inf,True,True,False,3.942686e+07,38218604,1.208256e+06,1.208256e+06,0.031614,0.031614,True
1215,state_and_local_tax_deductions,All,0.0,5000.0,False,False,False,1.086803e+09,334608000,7.521951e+08,7.521951e+08,2.247989,2.247989,True
1216,state_and_local_tax_deductions,All,0.0,5000.0,True,False,False,2.910782e+05,246102,4.497616e+04,4.497616e+04,0.182754,0.182754,True
1217,state_and_local_tax_deductions,All,5000.0,10000.0,False,False,False,1.242429e+09,254813000,9.876161e+08,9.876161e+08,3.875847,3.875847,True
1218,state_and_local_tax_deductions,All,5000.0,10000.0,True,False,False,3.470903e+05,320279,2.681135e+04,2.681135e+04,0.083712,0.083712,True
1219,state_and_local_tax_deductions,All,10000.0,15000.0,False,False,False,2.074838e+09,484707000,1.590131e+09,1.590131e+09,3.280603,3.280603,False
1220,state_and_local_tax_deductions,All,10000.0,15000.0,True,False,False,6.054434e+05,558434,4.700944e+04,4.700944e+04,0.084181,0.084181,True


In [4]:
# What happened over reweighting?

from IPython.display import Markdown

soi_from_tmd_2021["Improved"] = (
    soi_from_tmd_2021["Absolute error"].values
    < soi_from_tc_puf_2021["Absolute error"].values
)

pct_improved = soi_from_tmd_2021["Improved"].mean()

Markdown(
    f"**{pct_improved:.1%}** of SOI statistics improved over reweighting."
)

**53.6%** of SOI statistics improved over reweighting.

## SOI aggregates by variable

The below table shows the SOI statistics (and how TMD compares) for all SOI variables in aggregate (all-filer-statistics, total and counts).

In [5]:
# Show all rows

pd.set_option("display.max_rows", None)

soi_from_tmd_2021[soi_from_tmd_2021["Full population"]].sort_values(
    ["Variable"]
).drop(
    columns=[
        "Full population",
        "Filing status",
        "AGI lower bound",
        "AGI upper bound",
        "Taxable only",
    ]
)

,Variable,Count,Value,SOI Value,Error,Absolute error,Relative error,Absolute relative error,OK,Improved
2,adjusted_gross_income,False,1.442415e+13,14795614070000,-3.714629e+11,3.714629e+11,-0.025106,0.025106,True,True
190,business_net_losses,False,1.374200e+11,105580403000,3.183963e+10,3.183963e+10,0.301568,0.301568,False,True
192,business_net_losses,True,6.307588e+06,7546660,-1.239072e+06,1.239072e+06,-0.164188,0.164188,False,False
262,business_net_profits,False,6.928821e+11,517081772000,1.758003e+11,1.758003e+11,0.339985,0.339985,False,True
264,business_net_profits,True,1.996657e+07,21105685,-1.139113e+06,1.139113e+06,-0.053972,0.053972,False,False
334,capital_gains_distributions,False,2.429606e+10,23889533000,4.065291e+08,4.065291e+08,0.017017,0.017017,True,True
336,capital_gains_distributions,True,4.658074e+06,4505544,1.525296e+05,1.525296e+05,0.033854,0.033854,True,True
406,capital_gains_gross,False,9.822276e+11,2048795356000,-1.066568e+12,1.066568e+12,-0.520583,0.520583,False,False
408,capital_gains_gross,True,1.421543e+07,20497375,-6.281943e+06,6.281943e+06,-0.306476,0.306476,False,False
478,capital_gains_losses,False,9.659124e+09,16241889000,-6.582765e+09,6.582765e+09,-0.405296,0.405296,False,False


## SALT distributions

In [6]:
soi_from_tmd_2021[soi_from_tmd_2021.Variable == "state_and_local_tax_deductions"].sort_values(["Count", "AGI lower bound"])

,Variable,Filing status,AGI lower bound,AGI upper bound,Count,Taxable only,Full population,Value,SOI Value,Error,Absolute error,Relative error,Absolute relative error,OK,Improved
1893,state_and_local_tax_deductions,All,-inf,inf,False,False,True,3.446108e+11,362507801000,-1.789703e+10,1.789703e+10,-0.049370,0.049370,True,False
1894,state_and_local_tax_deductions,All,-inf,inf,False,True,False,3.181439e+11,352542556000,-3.439866e+10,3.439866e+10,-0.097573,0.097573,False,False
1897,state_and_local_tax_deductions,All,0.0,5000.0,False,False,False,6.790094e+08,336650000,3.423594e+08,3.423594e+08,1.016959,1.016959,True,True
1899,state_and_local_tax_deductions,All,5000.0,10000.0,False,False,False,5.241661e+08,380502000,1.436641e+08,1.436641e+08,0.377565,0.377565,True,True
1901,state_and_local_tax_deductions,All,10000.0,15000.0,False,False,False,7.431317e+08,581150000,1.619817e+08,1.619817e+08,0.278726,0.278726,True,True
1903,state_and_local_tax_deductions,All,15000.0,20000.0,False,False,False,8.102361e+08,821023000,-1.078687e+07,1.078687e+07,-0.013138,0.013138,True,True
1905,state_and_local_tax_deductions,All,20000.0,25000.0,False,False,False,1.014731e+09,811022000,2.037086e+08,2.037086e+08,0.251175,0.251175,True,True
1907,state_and_local_tax_deductions,All,25000.0,30000.0,False,False,False,1.112750e+09,835417000,2.773325e+08,2.773325e+08,0.331969,0.331969,True,True
1909,state_and_local_tax_deductions,All,30000.0,35000.0,False,False,False,1.409507e+09,1209086000,2.004206e+08,2.004206e+08,0.165762,0.165762,True,False
1911,state_and_local_tax_deductions,All,35000.0,40000.0,False,False,False,1.701400e+09,1372777000,3.286227e+08,3.286227e+08,0.239385,0.239385,True,True
